In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

tb_logdir = "/tmp/tf_tutorial/05_DenseNeuralNetwork"

#Data parameters
image_size = 28*28
n_class = 10

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#Trainin parameters
learning_rate = 0.1
total_steps = 1000
batch_size = 128
dense_units = [128, 256]

X = tf.placeholder(tf.float32, shape=[None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, shape=[None, n_class], name='Label')

In [3]:
#Two ways of having a dense layer.
def neural_net(self, shape, scope):
    with tf.name_scope(scope):
        W = tf.Variable(tf.random_normal(shape, dtype=tf.float32), name='weight')
        b = tf.Variable(tf.random_normal([shape[1]]), name='bias')
        self = tf.add(tf.matmul(self, W), b)
    return self

def neural_net_ez(self, units, name):
    self = tf.layers.dense(self, units=units, name=name)
    return self

In [4]:
with tf.name_scope('Model'):
    dense_net = neural_net(X, (image_size, dense_units[0]), 'dense_layer1')
    dense_net = neural_net(dense_net, (dense_units[0], dense_units[1]), 'dense_layer2')
    dense_net = neural_net(dense_net, (dense_units[1], n_class), 'dense_layer3')

In [5]:
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_net, labels=Y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    correct_pred =  tf.equal(tf.argmax(dense_net, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
init = tf.global_variables_initializer()
input_image = tf.reshape(X, [-1, 28, 28, 1])
tf.summary.image("input_image", input_image)
tf.summary.scalar('loss', loss_op)
tf.summary.scalar('accuracy', accuracy)
merge_summary_op = tf.summary.merge_all()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(tb_logdir, graph=tf.get_default_graph())
    for step in range(total_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        summary, acc, _ = sess.run([merge_summary_op, accuracy, train_op], feed_dict={X:batch_x, Y:batch_y})
        train_writer.add_summary(summary, step)
        if step%100 == 0:
            print("step: {}  accuracy:{:.2f}%".format(step, acc*100))
    print('Training Complete!')
    train_acc = sess.run(accuracy, feed_dict={X:mnist.train.images, Y:mnist.train.labels})
    print("Final Training acc:{:.2f}%".format(train_acc*100))
    test_acc = sess.run(accuracy, feed_dict={X:mnist.test.images[:3000], Y:mnist.test.labels[:3000]})
    print("Testing acc       :{:.2f}%".format(test_acc*100))

step: 0  accuracy:10.16%
step: 100  accuracy:82.03%
step: 200  accuracy:81.25%
step: 300  accuracy:85.94%
step: 400  accuracy:75.78%
step: 500  accuracy:80.47%
step: 600  accuracy:86.72%
step: 700  accuracy:82.81%
step: 800  accuracy:90.62%
step: 900  accuracy:84.38%
Training Complete!
Final Training acc:85.48%
Testing acc       :83.10%
